In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data/synthesized/train_data.npy');
path_to_labels = path.join(root_folder,'data/train_data/synthesized/train_labels.npy');
path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 25;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)




In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/7942544 (0%)]	Loss: 0.93659418821334838867
Train Epoch: 0 [450000/7942544 (6%)]	Loss: 0.24551975727081298828
Train Epoch: 0 [900000/7942544 (11%)]	Loss: 0.16494670510292053223
Train Epoch: 0 [1350000/7942544 (17%)]	Loss: 0.11896824836730957031
Train Epoch: 0 [1800000/7942544 (23%)]	Loss: 0.09128692746162414551
Train Epoch: 0 [2250000/7942544 (28%)]	Loss: 0.07218323647975921631
Train Epoch: 0 [2700000/7942544 (34%)]	Loss: 0.06267103552818298340
Train Epoch: 0 [3150000/7942544 (40%)]	Loss: 0.05827458202838897705
Train Epoch: 0 [3600000/7942544 (45%)]	Loss: 0.04927271604537963867
Train Epoch: 0 [4050000/7942544 (51%)]	Loss: 0.04851565510034561157
Train Epoch: 0 [4500000/7942544 (57%)]	Loss: 0.04520235583186149597
Train Epoch: 0 [4950000/7942544 (62%)]	Loss: 0.04216259345412254333
Train Epoch: 0 [5400000/7942544 (68%)]	Loss: 0.04043296724557876587
Train Epoch: 0 [5850000/7942544 (74%)]	Loss: 0.04126227274537086487
Train Epoch: 0 [6300000/7942544 (79%)]	Loss: 0.04160914942

Train Epoch: 6 [2250000/7942544 (28%)]	Loss: 0.02922589145600795746
Train Epoch: 6 [2700000/7942544 (34%)]	Loss: 0.03221332281827926636
Train Epoch: 6 [3150000/7942544 (40%)]	Loss: 0.03285795822739601135
Train Epoch: 6 [3600000/7942544 (45%)]	Loss: 0.03320379182696342468
Train Epoch: 6 [4050000/7942544 (51%)]	Loss: 0.03004596754908561707
Train Epoch: 6 [4500000/7942544 (57%)]	Loss: 0.03030888177454471588
Train Epoch: 6 [4950000/7942544 (62%)]	Loss: 0.03381177410483360291
Train Epoch: 6 [5400000/7942544 (68%)]	Loss: 0.03427452966570854187
Train Epoch: 6 [5850000/7942544 (74%)]	Loss: 0.03335095196962356567
Train Epoch: 6 [6300000/7942544 (79%)]	Loss: 0.02625917643308639526
Train Epoch: 6 [6750000/7942544 (85%)]	Loss: 0.02849790267646312714
Train Epoch: 6 [7200000/7942544 (91%)]	Loss: 0.03122756071388721466
Train Epoch: 6 [7650000/7942544 (96%)]	Loss: 0.03067571669816970825

Test set: Average loss: 0.00000185259386853431, Accuracy: 1966435/1985636 (99.03300504221317623887%)

Train Epoch: 

Train Epoch: 12 [4050000/7942544 (51%)]	Loss: 0.02909808419644832611
Train Epoch: 12 [4500000/7942544 (57%)]	Loss: 0.03306994959712028503
Train Epoch: 12 [4950000/7942544 (62%)]	Loss: 0.03273532167077064514
Train Epoch: 12 [5400000/7942544 (68%)]	Loss: 0.03187110647559165955
Train Epoch: 12 [5850000/7942544 (74%)]	Loss: 0.02983328513801097870
Train Epoch: 12 [6300000/7942544 (79%)]	Loss: 0.03527905046939849854
Train Epoch: 12 [6750000/7942544 (85%)]	Loss: 0.02692689001560211182
Train Epoch: 12 [7200000/7942544 (91%)]	Loss: 0.02913604490458965302
Train Epoch: 12 [7650000/7942544 (96%)]	Loss: 0.03066858649253845215

Test set: Average loss: 0.00000185177430012118, Accuracy: 1966487/1985636 (99.03562385049424676708%)

Train Epoch: 13 [0/7942544 (0%)]	Loss: 0.03036973066627979279
Train Epoch: 13 [450000/7942544 (6%)]	Loss: 0.03066744469106197357
Train Epoch: 13 [900000/7942544 (11%)]	Loss: 0.03289855644106864929
Train Epoch: 13 [1350000/7942544 (17%)]	Loss: 0.03075315803289413452
Train Epoc

Train Epoch: 18 [5400000/7942544 (68%)]	Loss: 0.03397882729768753052
Train Epoch: 18 [5850000/7942544 (74%)]	Loss: 0.03258402645587921143
Train Epoch: 18 [6300000/7942544 (79%)]	Loss: 0.03073626011610031128
Train Epoch: 18 [6750000/7942544 (85%)]	Loss: 0.02848379500210285187
Train Epoch: 18 [7200000/7942544 (91%)]	Loss: 0.03447763994336128235
Train Epoch: 18 [7650000/7942544 (96%)]	Loss: 0.02848874591290950775

Test set: Average loss: 0.00000185114834039268, Accuracy: 1966461/1985636 (99.03431444635371860841%)

Train Epoch: 19 [0/7942544 (0%)]	Loss: 0.03278088942170143127
Train Epoch: 19 [450000/7942544 (6%)]	Loss: 0.02919267117977142334
Train Epoch: 19 [900000/7942544 (11%)]	Loss: 0.03399485349655151367
Train Epoch: 19 [1350000/7942544 (17%)]	Loss: 0.02758751809597015381
Train Epoch: 19 [1800000/7942544 (23%)]	Loss: 0.02601863630115985870
Train Epoch: 19 [2250000/7942544 (28%)]	Loss: 0.02726941928267478943
Train Epoch: 19 [2700000/7942544 (34%)]	Loss: 0.03311435878276824951
Train Epoc

Train Epoch: 24 [6750000/7942544 (85%)]	Loss: 0.02707310952246189117
Train Epoch: 24 [7200000/7942544 (91%)]	Loss: 0.02926854416728019714
Train Epoch: 24 [7650000/7942544 (96%)]	Loss: 0.03027512133121490479

Test set: Average loss: 0.00000185099440841441, Accuracy: 1966448/1985636 (99.03365974428344031821%)



In [7]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00000185099440841441, Accuracy: 1966448/1985636 (99.03365974428344031821%)

